In [1]:
%run ../../scripts/bios_utils.py
%run ../escher_grid_merge.py
%run ../escher_factory_api.py

In [2]:
import escher
import cobra
import modelseed_escher
from modelseed_escher.map import EscherCluster

In [3]:
escher_manager = modelseed_escher.EscherManager(escher)

In [5]:
data = read_json('/Users/fliu/workspace/jupyter/data/www/annotation/data/latest_cluster.json')
em = modelseed_escher.core.EscherMap(data['escher_map'])
em_grid = data['grid']

In [6]:
#em.display_in_notebook()

'/Users/fliu/Library/Caches/escher/1-0-0/5'

In [8]:
em_layer = add_layers(em.clone(), em_grid, escher_manager)

150 ModelSEED.Ribitol et al Utilization 0 0 4500 2427 c


In [9]:
#em_layer.display_in_notebook()

In [10]:
ec = EscherCluster(25)
cluster_data = ec.cluster(em_layer)
result = report(cluster_data, em_layer)
result = sort_by_database(result, 'seed.compound')

In [54]:
builder = escher.Builder(map_json=json.dumps(em.escher_map))

In [93]:
import logging
logger = logging.getLogger(__name__)
def report(cluster_data, em):
    cluster_map = em.escher_map
    any_merge = True
    result = {
        'models' : set(),
        'records' : []
    }
    for cluster in cluster_data:
        logger.debug("%s", cluster)
        record = {
            'model_data' : {},
            'database' : {}
        }
        match_cmp = '?'
        cmps = set()
        for id in cluster:
            node_id = cluster_map[1]['nodes'][id]['bigg_id']
            logger.debug("%s", node_id)
            if node_id.startswith('cpd') and '_' in node_id:
                cpd_id, cmp = node_id.split('_')
                cmps.add(cmp)

        if len(cmps) == 1:
            match_cmp = cmps.pop()
        for id in cluster:
            node_id = cluster_map[1]['nodes'][id]['bigg_id']
            cpd_id = node_id
            database = None
            if '@' in node_id:
                cpd_id, database = node_id.split('@')

            #print(cpd_id, database)
            #if is model
            if not database == None:
                result['models'].add(database)
                cmp = match_cmp
                if not cmp in record['model_data']:
                    record['model_data'][cmp] = {}
                if not database in record['model_data'][cmp]:
                    record['model_data'][cmp][database] = set()
                record['model_data'][cmp][database].add(cpd_id)
            else:
                database = 'seed.compound'
                if not database in record['database']:
                    record['database'][database] = set()
                if node_id.startswith('cpd') and '_' in node_id:
                    cpd_id, cmp = node_id.split('_')
                record['database'][database].add(cpd_id)
            #elseif database
        if len(record['model_data']) > 0:
            result['records'].append(record)


    result['models'] = list(result['models'])
    for r in result['records']:
        for cmp_id in r['model_data']:
            for model_id in r['model_data'][cmp_id]:
                r['model_data'][cmp_id][model_id] = list(r['model_data'][cmp_id][model_id])
        for database_id in r['database']:
            r['database'][database_id] = list(r['database'][database_id])
            
    return result
dd = report(cluster_data, em_layer)



In [94]:
dd['records'][0]

{'model_data': {'c': {'iMM904': ['M_nadp_c']}},
 'database': {'seed.compound': ['cpd00006']}}

In [1]:
def sort_by_database(report, db):
    result = {
        'models' : report['models']
    }
    m = {}
    records = []
    for r in report['records']:
        if 'seed.compound' in r['database']:
            key = tuple(sorted(r['database']['seed.compound']))
            if not key in m:
                m[key] = {}
            for cmp in r['model_data']:
                if not cmp in m[key]:
                    m[key][cmp] = {}
                for model_id in r['model_data'][cmp]:
                    if not model_id in m[key][cmp]:
                        m[key][cmp][model_id] = set()
                    m[key][cmp][model_id] |= set(r['model_data'][cmp][model_id])
        else:
            records.append(r)
    for key in m:
        record = {
            'model_data' : {},
            'database' : {db : list(key)}
        }
        for cmp in m[key]:
            record[cmp] = {}
            for model_id in m[key][cmp]:
                record[cmp][model_id] = list(m[key][cmp][model_id])
        records.append(record)
        print(record)
    result['records'] = records
    return report
    
#sort_by_database(dd, 'seed.compound')


In [11]:
import escher
escher_manager = modelseed_escher.EscherManager(escher)

!!! cpd00011


In [72]:
em = escher_manager.get_map('ModelSEED', 'ModelSEED', 'Central_Metabolism_JE')
uids = em.add_uid_to_reaction_metabolites()
def move_to_compartment(cmp_id, em):
    em.add_uid_to_reaction_metabolites()
    node_uid_cmp = {}
    node_uid_id = {}
    for node_uid in em.escher_graph['nodes']:
        n = em.escher_graph['nodes'][node_uid]
        if n['node_type'] == 'metabolite':
            if not 'compartment' in n:
                node_uid_cmp[node_uid] = cmp_id
                n['bigg_id'] += '_' + cmp_id
                node_uid_id[node_uid] = n['bigg_id']
            else:
                node_uid_id[node_uid] = n['bigg_id']
    add_compartment(em, node_uid_cmp)
    for rxn_uid in em.escher_graph['reactions']:
        rnode = em.escher_graph['reactions'][rxn_uid]
        for o in rnode['metabolites']:
            o['bigg_id'] = node_uid_id[o['node_uid']]
            
        rnode['bigg_id'] += '_' + cmp_id
    return node_uid_cmp
cmp = move_to_compartment('z', em)

!!! cpd00011
!!! cpd00011


In [ ]:
ls 

In [61]:
em = None
with open('/Users/fliu/workspace/jupyter/data/escher/maps/ModelSEED/ModelSEED.Central_Metabolism_JE.json', 'r') as fh:
    em = json.loads(fh.read())

In [62]:
for uid in em[1]['nodes']:
    n = em[1]['nodes'][uid]
    if n['node_type'] == 'metabolite':
        if n['bigg_id'].endswith('_e'):
            n['compartment'] = 'e'

In [63]:
with open('/Users/fliu/workspace/jupyter/data/escher/maps/ModelSEED/ModelSEED.Central_Metabolism_JE.json', 'w') as fh:
    fh.write(json.dumps(em))

In [65]:
build_data = {"maps":["iMM904;c;c;ModelSEED.Central_Metabolism_JE"],"x":1,"y":1}
map_assembly = process_build_data_input(build_data['maps'])
grid_x = build_data['x']
grid_y = build_data['y']
print(map_assembly)

[{'map_id': 'Central_Metabolism_JE', 'cmp_target': ['c'], 'cmp_sbml': ['c'], 'sbml_id': 'iMM904'}]


In [22]:
build_data = {"maps":["iMM904;c;c;ModelSEED.Tryptophan Biosynthesis"],"x":1,"y":1}
map_assembly = process_build_data_input(build_data['maps'])
grid_x = build_data['x']
grid_y = build_data['y']
print(map_assembly)

[{'map_id': 'Tryptophan Biosynthesis', 'cmp_target': ['c'], 'cmp_sbml': ['c'], 'sbml_id': 'iMM904'}]


### Loading data from BIOS server, cache files and setup EscherFactoryApi

In [4]:
import biosapi
from biosapi import BIOS
from biosapi.bios_model_alias_mapper import BiosModelAliasMapper
bios = BIOS('http://192.168.1.15:8080/biosynth-web-biobase')

False

In [46]:
MODEL_CPD_MAPPING = {}
MODEL_RXN_MAPPING = {}
MODEL_CMP_MAPPING = {}
with open('../data/cpd_mapping_cache4.json', 'r') as f:
    MODEL_CPD_MAPPING = json.loads(f.read())
with open('../data/rxn_mapping_cache4.json', 'r') as f:
    MODEL_RXN_MAPPING = json.loads(f.read())
with open('../data/cmp_mapping_cache.json', 'r') as f:
    MODEL_CMP_MAPPING = json.loads(f.read())

In [47]:
%run ../bios_mock.py
bios = BIOS_MOCK('../data/bios_cache_fungi.json')
escher_factory = EscherFactoryApi(escher_manager)
escher_factory.model_path = '/Users/fliu/workspace/jupyter/python3/annotation-server/data/'
escher_factory.cpd_mapping = MODEL_CPD_MAPPING
escher_factory.rxn_mapping = MODEL_RXN_MAPPING
escher_factory.cmp_mapping = MODEL_CMP_MAPPING
escher_factory.bios_cache = bios.model_data
#escher_factory.cpd_mapping['core'] = wut_map
#escher_factory.rxn_mapping['core'] = wut_rxn_map

In [26]:
with open('../data/cpd_mapping_cache4.json', 'w') as f:
    f.write(json.dumps(MODEL_CPD_MAPPING))

In [6]:
model_id = 'iMM904'
mapper = BiosModelAliasMapper(bios)
mapper.load_model(model_id)
rxn_mapping = mapper.get_rxn_mapping('ModelSeedReaction')
for rxn_id in MODEL_RXN_MAPPING[model_id]:
    if rxn_id in rxn_mapping:
        for bios_alias in rxn_mapping[rxn_id]:
            if not bios_alias in MODEL_RXN_MAPPING[model_id][rxn_id]:
                MODEL_RXN_MAPPING[model_id][rxn_id].append(bios_alias)

In [7]:
model_id = 'iJDZ836'
mapper = BiosModelAliasMapper(bios)
mapper.load_model(model_id)
rxn_mapping = mapper.get_rxn_mapping('ModelSeedReaction')
for rxn_id in MODEL_RXN_MAPPING[model_id]:
    if rxn_id in rxn_mapping:
        for bios_alias in rxn_mapping[rxn_id]:
            if not bios_alias in MODEL_RXN_MAPPING[model_id][rxn_id]:
                MODEL_RXN_MAPPING[model_id][rxn_id].append(bios_alias)

In [97]:
core_model = None
with open('/Users/fliu/workspace/jupyter/data/escher/models/ModelSEED/Core.json', 'r') as fh:
    core_model = json.loads(fh.read())

In [155]:
wut_map = {}
for o in core_model['metabolites']:
    #print(o['id'], o['compartment'])
    if not o['compartment'] in wut_map:
        wut_map[o['compartment']] = {}
    wut_map[o['compartment']][o['id']] = [o['id']]

In [177]:
wut_rxn_map = {}
for o in core_model['reactions']:
    wut_rxn_map[o['id']] = [o['id']]

In [141]:
#escher_factory.get_cpd_mapping('iMM904','c')
#{'cpd00201': ['M_10fthf_c'],

In [9]:
build_data = {"maps":["core;c;c;ModelSEED.Central_Metabolism_JE"],"x":1,"y":1}
map_assembly = process_build_data_input(build_data['maps'])
grid_x = build_data['x']
grid_y = build_data['y']
print(map_assembly)

[{'map_id': 'Central_Metabolism_JE', 'cmp_target': ['c'], 'cmp_sbml': ['c'], 'sbml_id': 'core'}]


In [ ]:
MODEL_CPD_MAPPING['core'] = wut_map

In [179]:
em_list = []
for ma in map_assembly:
    escher_model_id = 'ModelSEED'
    escher_map_id = ma['map_id']
    sbml_id = ma['sbml_id']
    target_cmp = ma['cmp_target'][0]
    cmp_sbml = ma['cmp_sbml'][0]
    cpd_mapping = escher_factory.get_cpd_mapping('core','c')
    rxn_mapping = escher_factory.get_rxn_mapping('core')
    #em = self.escher_manager.get_map('ModelSEED', escher_model_id, escher_map_id)

In [166]:
escher_factory.cpd_match_exclude

set()

In [169]:
f = escher_factory.lambda_get_model_reaction('core')

In [170]:
f

<function EscherFactoryApi.lambda_get_model_reaction.<locals>.<lambda>(x)>

In [181]:
em = escher_manager.get_map('ModelSEED', escher_model_id, escher_map_id)
em.add_uid_to_reaction_metabolites()
a = move_to_compartment(target_cmp, em)
def remap_map_compounds(em, bigg_to_seed, id_function = lambda x : (x, 0)):
    #print(bigg_to_seed)
    map_compound_remap = {}
    unmaped = set()
    node_uid_cmp = {}
    for map_uid in em.escher_graph['nodes']:
        node = em.escher_graph['nodes'][map_uid]
        if node['node_type'] == 'metabolite':
            node_id = node['bigg_id']
            #print(node_id, node)
            bigg_id, cmp = id_function(node_id)
            #print(node_id, bigg_id, cmp)
            #bigg_id = node_id[:-2]
            #cmp = node_id[-1:]
            
            #print(node_id, bigg_id, cmp)
            if bigg_id in bigg_to_seed:
                node_uid_cmp[map_uid] = cmp
                map_compound_remap[node_id] = set(bigg_to_seed[bigg_id])
                #print(map_uid, bigg_id, cmp, bigg_to_seed[bigg_id])
            else:
                unmaped.add(node_id)
    return map_compound_remap, unmaped, node_uid_cmp
map_compound_remap, unmaped_cpd, node_uid_cmp = remap_map_compounds(em, cpd_mapping, lambda x : x.split('_'))
logger.warning('unmaped_cpd: %d', len(unmaped_cpd))
map_reaction_remap, unmaped_rxn = remap_map_reactions(em, rxn_mapping, map_compound_remap, escher_factory.cpd_match_exclude,
                                                              escher_factory.lambda_get_model_reaction(sbml_id),
                                                              lambda x : x, lambda x : x[:-2])
logger.warning('unmaped_rxn: %d', len(unmaped_rxn))

print(sbml_id, map_reaction_remap)

cpd_remap = {}
rxn_remap = {}
for k in map_compound_remap:
    cpd_remap[k] = list(map_compound_remap[k])[0] + '@' + sbml_id
for k in map_reaction_remap:
    rxn_remap[k] = list(map_reaction_remap[k])[0] + '@' + sbml_id
#print(sbml_id, rxn_remap)
em.swap_ids(cpd_remap, rxn_remap)
em.delete_reactions(unmaped_rxn)
em.delete_metabolites(unmaped_cpd)

!!! cpd00011
!!! cpd00011
core {}


In [ ]:
def translate_to_model(self, escher_model_id, escher_map_id, sbml_id, target_cmp, cpd_mapping, rxn_mapping):
        em = self.escher_manager.get_map('ModelSEED', escher_model_id, escher_map_id)
        em.add_uid_to_reaction_metabolites()

        move_to_compartment(target_cmp, em)

        map_compound_remap, unmaped_cpd, node_uid_cmp = remap_map_compounds(em, 
                                                                            cpd_mapping, lambda x : x.split('_'))

        logger.warning('unmaped_cpd: %d', len(unmaped_cpd))

        map_reaction_remap, unmaped_rxn = remap_map_reactions(em, rxn_mapping, map_compound_remap, self.cpd_match_exclude,
                                                              self.lambda_get_model_reaction(sbml_id),
                                                              lambda x : x, lambda x : x[:-2])

        logger.warning('unmaped_rxn: %d', len(unmaped_rxn))

        print(sbml_id, map_reaction_remap)
        
        cpd_remap = {}
        rxn_remap = {}
        for k in map_compound_remap:
            cpd_remap[k] = list(map_compound_remap[k])[0] + '@' + sbml_id
        for k in map_reaction_remap:
            rxn_remap[k] = list(map_reaction_remap[k])[0] + '@' + sbml_id
        #print(sbml_id, rxn_remap)
        em.swap_ids(cpd_remap, rxn_remap)
        em.delete_reactions(unmaped_rxn)
        em.delete_metabolites(unmaped_cpd)
        return em

In [61]:
for uid in em.escher_graph['nodes']:
    n = em.escher_graph['nodes'][uid]
    if n['node_type'] == 'metabolite':
        cpd_id, cmp = n['bigg_id'].split('_')
        if not 'compartment' in n:
            n['compartment'] = cmp
        #print(n['bigg_id'], n['compartment'])
for uid in em.escher_graph['reactions']:
    n = em.escher_graph['reactions'][uid]
    rxn_id, cmp = n['bigg_id'].split('_')
    if not 'compartment' in n:
        n['compartment'] = cmp
#with open('/Users/fliu/workspace/jupyter/web/annotation/data/TempModels/test_fungi.json', 'w') as fh:
#    fh.write(json.dumps(em.escher_map))

In [12]:
target_cmp = 'c'
em = None
with open('/Users/fliu/workspace/jupyter/web/annotation/data/TempModels/test_fungi.json', 'r') as fh:
    em = modelseed_escher.core.EscherMap(json.loads(fh.read()))
em.add_uid_to_reaction_metabolites()
a = move_to_compartment(target_cmp, em)

In [9]:
build_data = {"maps":["iMM904;c;c;ModelSEED.Central_Metabolism_JE"],"x":1,"y":1}
map_assembly = process_build_data_input(build_data['maps'])
grid_x = build_data['x']
grid_y = build_data['y']
em = escher_factory.build_grid(map_assembly, (1,1))

translate_to_model: iMM904 [c > c]


Central_Metabolism_JE
!!! cpd00011
!!! cpd00011


unmaped_cpd: 82
unmaped_rxn: 157
R_PPCK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PYK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PFK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_HEX1@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_FBP@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PC@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_G6PDH2r@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_G6PI@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_BTDt_RR@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_FDH@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_ACt2r@iMM904 contains LR/RL direction
R_ALCD2ir@iMM904 contains LR/RL direction
R_ACACT1r@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PGL@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_GND@iMM904 incorr

R_ACS@iMM904
map  : {'M_atp_c@iMM904': -1, 'M_coa_c@iMM904': -1, 'M_ac_c@iMM904': -1, 'M_ppi_c@iMM904': 1, 'M_amp_c@iMM904': 1, 'M_accoa_c@iMM904': 1, 'M_h_c@iMM904': 1}
model: {'M_amp_c@iMM904': 1.0, 'M_ppi_c@iMM904': 1.0, 'M_accoa_c@iMM904': 1.0, 'M_coa_c@iMM904': -1.0, 'M_ac_c@iMM904': -1.0, 'M_atp_c@iMM904': -1.0}
R_SULR@iMM904
map  : {'M_h2o_c@iMM904': -3, 'M_nadp_c@iMM904': -3, 'M_h2s_c@iMM904': -1, 'M_nadph_c@iMM904': 3, 'M_h_c@iMM904': 3, 'M_so3_c@iMM904': 1}
model: {'M_nadp_c@iMM904': 3.0, 'M_h2s_c@iMM904': 1.0, 'M_h2o_c@iMM904': 3.0, 'M_so3_c@iMM904': -1.0, 'M_nadph_c@iMM904': -3.0, 'M_h_c@iMM904': -5.0}


In [10]:
build_data = {"maps":["iMM904;c;c;ModelSEED.Fungi_Central_Temp"],"x":1,"y":1}
map_assembly = process_build_data_input(build_data['maps'])
grid_x = build_data['x']
grid_y = build_data['y']
em = escher_factory.build_grid(map_assembly, (1,1))

translate_to_model: iMM904 [c > c]
unmaped_cpd: 29
unmaped_rxn: 86


Fungi_Central_Temp


R_PPCK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_ME2m@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_ALDD2ym@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PDHm@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_GLCt1@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PYK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_ICDHxm@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_ALDD2xm@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PPAm@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PFK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PYRDC@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_HEX1@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_FRUK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_FRDm@iMM904 contains LR/RL direction
R_ME1m@iMM904 inc

In [12]:
model = None
with open('/Users/fliu/workspace/jupyter/web/annotation/data/TempModels/iMM904.json', 'r') as fh:
    model = cobra.io.from_json(fh.read())

In [42]:
aaaa = 'M_AMMONIA'
for k in escher_factory.cpd_mapping['iJDZ836']:
    if aaaa in escher_factory.cpd_mapping['iJDZ836'][k]:
        print(k, escher_factory.cpd_mapping['iJDZ836'][k][aaaa])

CCO__45__CYTOSOL ['cpd00013']


In [45]:

cpd_mapping = escher_factory.get_cpd_mapping(sbml_id)
cpd_mapping['CCO__45__CYTOSOL']['cpd00001']

['M_WATER']

In [48]:
escher_model_id = 'ModelSEED'
escher_map_id = 'Fungi_Central_Temp'
sbml_id = 'iJDZ836'
target_cmp = 'c'
cmp_mapping = escher_factory.get_cmp_mapping(sbml_id)
cpd_mapping = escher_factory.get_cpd_mapping(sbml_id)
rxn_mapping = escher_factory.get_rxn_mapping(sbml_id)

em = escher_factory.translate_to_model2(escher_model_id, escher_map_id, sbml_id, target_cmp, 
                                        cpd_mapping, rxn_mapping, cmp_mapping)
#WARNING:__main__:unmaped_cpd: 78
#WARNING:__main__:unmaped_rxn: 118

In [49]:
em.display_in_notebook()

In [82]:
escher_model_id = 'ModelSEED'
escher_map_id = 'Fungi_Central_Temp'
sbml_id = 'iAL1006'
target_cmp = 'c'
cmp_mapping = escher_factory.cmp_mapping[sbml_id]
cpd_mapping = {}
for model_cmp in cmp_mapping.values():
    cpd_mapping[model_cmp] = escher_factory.get_cpd_mapping(sbml_id, model_cmp)
rxn_mapping = escher_factory.get_rxn_mapping(sbml_id)

em = escher_factory.translate_to_model2(escher_model_id, escher_map_id, sbml_id, target_cmp, 
                                        cpd_mapping, rxn_mapping, cmp_mapping)
#WARNING:__main__:unmaped_cpd: 34
#WARNING:__main__:unmaped_rxn: 104

iAL1006 {'rxn08428_e': {'R_r1426'}, 'rxn00247_c': {'R_r0018'}, 'rxn00799_m': {'R_r0097'}, 'rxn00161_m': {'R_r0058'}, 'rxn00459_c': {'R_r0015'}, 'rxn08617_e': {'R_r1342'}, 'rxn09777_m': {'R_r1182'}, 'rxn00148_c': {'R_r0016'}, 'rxn05467_m': {'R_r1246'}, 'rxn00248_m': {'R_r0098'}, 'rxn01100_c': {'R_r0013'}, 'rxn00505_m': {'R_r0088'}, 'rxn01106_c': {'R_r0014'}, 'rxn00248_c': {'R_r0099'}, 'rxn00506_m': {'R_r0053'}, 'rxn00001_m': {'R_r0115'}, 'rxn00168_c': {'R_r0046'}, 'rxn05467_e': {'R_r1424'}, 'rxn00285_m': {'R_r0095'}, 'rxn00288_m': {'R_r0096'}, 'rxn00175_m': {'R_r0026'}, 'rxn00175_c': {'R_r0025'}, 'rxn00781_c': {'R_r0012'}, 'rxn00256_m': {'R_r0085'}, 'rxn00198_m': {'R_r0089'}, 'rxn00747_c': {'R_r0011'}, 'rxn09739_m': {'R_r1231'}, 'rxn09273_m': {'R_r1239'}, 'rxn08063_m': {'R_r1244'}, 'rxn09713_m': {'R_r1232'}, 'rxn00507_c': {'R_r0054'}, 'rxn00001_c': {'R_r0114'}, 'rxn08428_m': {'R_r1247'}, 'rxn00543_c': {'R_r0020'}, 'rxn01115_c': {'R_r0029'}, 'rxn00543_m': {'R_r0021'}, 'rxn01116_c': {'R_r

In [12]:
build_data = {"maps":[
    "iMM904;c;c;ModelSEED.Fungi_Central_Temp",
    "iJDZ836;c;c;ModelSEED.Fungi_Central_Temp",
    "iAL1006;c;c;ModelSEED.Fungi_Central_Temp",
],"x":3,"y":1}
map_assembly = process_build_data_input(build_data['maps'])
grid_x = build_data['x']
grid_y = build_data['y']
em = escher_factory.build_grid(map_assembly, (grid_x,grid_y))

translate_to_model: iMM904 [c > c]
unmaped_cpd: 29
unmaped_rxn: 86


Fungi_Central_Temp


R_PPCK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_ME2m@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_ALDD2ym@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PDHm@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_GLCt1@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PYK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_ICDHxm@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_ALDD2xm@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PPAm@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PFK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_PYRDC@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_HEX1@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_FRUK@iMM904 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_FRDm@iMM904 contains LR/RL direction
R_ME1m@iMM904 inc

Fungi_Central_Temp


PEPCARBOXYKIN__45__RXN__45__L2R@iJDZ836 incorrect reversiblity: False, model: [-1000.00, 1000.00]
PEPDEPHOS__45__RXN__45__R2L@iJDZ836 incorrect reversiblity: False, model: [-1000.00, 1000.00]
RXN__45__8631__45__R2L@iJDZ836 contains LR/RL direction
RXN__45__6161__45__L2R@iJDZ836 incorrect reversiblity: False, model: [-1000.00, 1000.00]
ACETATE__45____45__COA__45__LIGASE__45__RXN__45__L2R@iJDZ836 incorrect reversiblity: False, model: [-1000.00, 1000.00]
GAPOXNPHOSPHN__45__RXN__45__R2L@iJDZ836 contains LR/RL direction
ALCOHOL__45__DEHYDROG__45__RXN__45__R2L@iJDZ836 contains LR/RL direction
RXN__45__9952__45__L2R@iJDZ836 incorrect reversiblity: False, model: [-1000.00, 1000.00]
GLUCOKIN__45__RXN__45__L2R@iJDZ836 incorrect reversiblity: False, model: [-1000.00, 1000.00]
ALDOSE__45__1__45__EPIMERASE__45__RXN__45__R2L@iJDZ836 contains LR/RL direction
RXN__45__1685__45__L2R@iJDZ836 incorrect reversiblity: False, model: [-1000.00, 1000.00]
PGLUCISOM__45__RXN__45__L2R@iJDZ836 incorrect reversibl

Fungi_Central_Temp


R_r1426@iAL1006 contains LR/RL direction
R_r0018@iAL1006 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_r0058@iAL1006 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_r1342@iAL1006 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_r0013@iAL1006 contains LR/RL direction
R_r0088@iAL1006 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_r0014@iAL1006 contains LR/RL direction
R_r1424@iAL1006 contains LR/RL direction
R_r0095@iAL1006 contains LR/RL direction
R_r0096@iAL1006 contains LR/RL direction
R_r0026@iAL1006 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_r0025@iAL1006 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_r0089@iAL1006 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_r0011@iAL1006 contains LR/RL direction
R_r1239@iAL1006 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_r1232@iAL1006 incorrect reversiblity: False, model: [-1000.00, 1000.00]
R_r0029@iAL1006 incorrect reversiblity: False, 

R_r0016@iAL1006
map  : {'cpd00067_c': -1, 'M_ATP@iAL1006': 1, 'M_PEP@iAL1006': -1, 'M_PYR@iAL1006': 1, 'M_ADP@iAL1006': -1}
model: {'M_ATP@iAL1006': 1.0, 'M_PYR@iAL1006': 1.0, 'M_ADP@iAL1006': -1.0, 'M_PEP@iAL1006': -1.0}
R_r0098@iAL1006
map  : {'M_OAm@iAL1006': 1, 'M_NADHm@iAL1006': 1, 'M_H_POm@iAL1006': 1, 'M_MALm@iAL1006': -1, 'M_NADm@iAL1006': -1}
model: {'M_NADHm@iAL1006': 1.0, 'M_OAm@iAL1006': 1.0, 'M_NADm@iAL1006': -1.0, 'M_MALm@iAL1006': -1.0}
R_r0099@iAL1006
map  : {'cpd00067_c': 1, 'M_NADH@iAL1006': 1, 'M_MAL@iAL1006': -1, 'M_NAD@iAL1006': -1, 'M_OA@iAL1006': 1}
model: {'M_NADH@iAL1006': 1.0, 'M_OA@iAL1006': 1.0, 'M_MAL@iAL1006': -1.0, 'M_NAD@iAL1006': -1.0}
R_r0053@iAL1006
map  : {'M_ACALm@iAL1006': -1, 'M_H_POm@iAL1006': 2, 'M_H2Om@iAL1006': -1, 'M_NADm@iAL1006': -1, 'M_NADHm@iAL1006': 1, 'M_ACm@iAL1006': 1}
model: {'M_NADHm@iAL1006': 1.0, 'M_ACm@iAL1006': 1.0, 'M_NADm@iAL1006': -1.0, 'M_H2Om@iAL1006': -1.0, 'M_ACALm@iAL1006': -1.0}
R_r0115@iAL1006
map  : {'M_H2Om@iAL1006':